### Загружаем DeepSort

In [ ]:
!git clone https://github.com/nwojke/deep_sort.git
!cd deep_sort && pip install -r /content/deep_sort/requirements-gpu.txt

Cloning into 'deep_sort'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 167 (delta 6), reused 13 (delta 6), pack-reused 144 (from 2)
Receiving objects: 100% (167/167), 84.99 KiB | 2.93 MiB/s, done.
Resolving deltas: 100% (86/86), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement tensorflow==2.10.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc

### Обучаем YOLO

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

data_yaml = """
train: /content/drive/MyDrive/Проекты/Мультитрекинг однородных объектов/YOLOv5-QCB+SORT/data/train/images
val: /content/drive/MyDrive/Проекты/Мультитрекинг однородных объектов/YOLOv5-QCB+SORT/data/val/images

nc: 1
names: ['bubble']
"""

data_path = '/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/предзащита/детекции_пузыри/data.yaml'

with open(data_path, 'w') as f:
    f.write(data_yaml)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import os
import subprocess
import time

start_time = time.time()

!yolo train data=/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/предзащита/детекции_пузыри/data.yaml model=/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/предзащита/yolo11s.pt epochs=100 batch=5 conf=0.4 max_det=1000 lr0=0.1 momentum=0.972

end_time = time.time()
print(f"Время тренировки: {end_time - start_time:.2f} секунд")

In [ ]:
def convert_yolo_to_mot(img_width, img_height, cx, cy, w, h):
    left = (cx - w / 2.0) * img_width
    top = (cy - h / 2.0) * img_height
    width = w * img_width
    height = h * img_height
    return (int(left), int(top), int(width), int(height))

class DeepSortTracker:
    def __init__(self, img_size,
        nms_max_overlap=0.6, max_cosine_distance=0.5, nn_budget=None):
        self.img_size = img_size
        self.nms_max_overlap = nms_max_overlap
        metric = nn_matching.NearestNeighborDistanceMetric(
            "cosine", max_cosine_distance, nn_budget)
        self.tracker = Tracker(metric)

    def prepare_detections(self, yolo_detections):
        img_width, img_height = self.img_size
        detections = []
        for line in yolo_detections:
            x1, y1, x2, y2, conf, cl  = line
            bbox = (x1, y1, x2 - x1, y2 - y1)
            feature = []
            detections.append(Detection(bbox, conf, feature))
        return detections

    def frame_callback(self, yolo_detections):
        detections = self.prepare_detections(yolo_detections)
        boxes = np.array([d.tlwh for d in detections])
        scores = np.array([d.confidence for d in detections])
        indices = preprocessing.non_max_suppression(
            boxes, self.nms_max_overlap, scores)
        detections = [detections[i] for i in indices]
        self.tracker.predict()
        self.tracker.update(detections)
        results = []
        for track in self.tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlwh()
            results.append([track.track_id, bbox[0], bbox[1], bbox[2], bbox[3]])
        return results

### Детекция с помощью YOLO

In [ ]:
import time
import cv2
import os
from ultralytics import YOLO
import numpy as np
from deep_sort.application_util import preprocessing
from deep_sort.application_util import visualization
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker

model = YOLO('/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/предзащита/best.pt')
video_path = '/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/предзащита/input_video.mp4'
detections_folder = '/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/предзащита/детекции_пузыри'
os.makedirs(detections_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Ошибка при открытии видео")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
output_path = '/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/предзащита/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
img_size = (frame_width, frame_height)
deepSortTracker = DeepSortTracker(img_size)
frame_number = 0
total_time = 0
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()
    results = model(frame, max_det=1000)
    end_time = time.time()
    inference_time = end_time - start_time
    total_time += inference_time
    frame_count += 1
    print(f"Кадр {frame_number}: время инференса = {inference_time:.4f} сек")

    annotations = results[0].boxes.data.cpu().numpy()
    if annotations.size > 0:
        detections_path = os.path.join(detections_folder, f"frame_{frame_number:05d}.txt")
        with open(detections_path, "w") as f:
            track_results = deepSortTracker.frame_callback(annotations)
            for track_box in track_results:
                track_id, x1, y1, width, height = track_box
                f.write(f"{int(track_id)},{x1},{y1},{width},{height}\n")
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x1 + width), int(y1 + height)), (0, 255, 0), 2)
                cv2.putText(frame, f'ID: {track_id}', (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    out.write(frame)
    frame_number += 1

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"\nОбщее время: {total_time:.4f} сек")
if frame_count > 0:
    avg_time = total_time / frame_count
    print(f"\nСреднее время инференса: {avg_time:.4f} сек/кадр\n")
    print(f"FPS инференса: {1 / avg_time:.2f} кадров/сек\n")
else:
    print("Не удалось обработать ни одного кадра.")
print(f"Детекции для каждого кадра сохранены в {detections_folder}")


0: 480x640 155 bubbles, 484.7ms
Speed: 6.7ms preprocess, 484.7ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)
Кадр 0: время инференса = 0.7566 сек

0: 480x640 147 bubbles, 446.3ms
Speed: 5.4ms preprocess, 446.3ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
Кадр 1: время инференса = 0.4585 сек

0: 480x640 133 bubbles, 427.1ms
Speed: 4.7ms preprocess, 427.1ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
Кадр 2: время инференса = 0.4384 сек

0: 480x640 138 bubbles, 421.7ms
Speed: 4.6ms preprocess, 421.7ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
Кадр 3: время инференса = 0.4318 сек

0: 480x640 145 bubbles, 418.9ms
Speed: 4.4ms preprocess, 418.9ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
Кадр 4: время инференса = 0.4294 сек

0: 480x640 142 bubbles, 416.6ms
Speed: 4.0ms preprocess, 416.6ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Кадр 5: время инференса = 0.4

### Преобразуем собственную разметку и DeepSORT + YOLOv11s в один формат

In [ ]:
import xml.etree.ElementTree as ET

def load_cvat_annotations(xml_path, max_frames=40, y_threshold=None):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    annotations = {}
    for track in root.findall("track"):
        track_id = int(track.get("id"))
        for box in track.findall("box"):
            frame = int(box.get("frame"))
            if frame >= max_frames:
                continue
            x1 = float(box.get("xtl"))
            y1 = float(box.get("ytl"))
            x2 = float(box.get("xbr"))
            y2 = float(box.get("ybr"))
            width = x2 - x1
            height = y2 - y1
            if y_threshold is None or (y1 + height) < y_threshold:
                if frame not in annotations:
                    annotations[frame] = []
                annotations[frame].append((track_id, x1, y1, width, height))
    return annotations

cvat_annotations = load_cvat_annotations("/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/предзащита/annotations.xml", max_frames=40, y_threshold=100)

In [ ]:
import os
import re

def load_deepsort_predictions(folder_path, max_frames=40, y_threshold=None):
    detections = {}

    for filename in sorted(os.listdir(folder_path)):
        match = re.match(r"frame_(\d+)\.txt", filename)
        if match:
            frame = int(match.group(1))
            if frame >= max_frames:
                continue

            detections[frame] = []
            with open(os.path.join(folder_path, filename), "r") as f:
                for line in f:
                    parts = line.strip().split(",")
                    if len(parts) != 5:
                        continue

                    track_id = int(parts[0])
                    x1 = float(parts[1])
                    y1 = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])

                    if y_threshold is None or (y1 + height) < y_threshold:
                        detections[frame].append((track_id, x1, y1, width, height))

    return detections

deepsort_detections = load_deepsort_predictions("/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/предзащита/детекции_пузыри", max_frames=40, y_threshold=100)

### Считаем показатель precision

In [ ]:
def iou(boxA, boxB, tolerance=0.7):
    x1A, y1A, wA, hA = boxA
    x1B, y1B, wB, hB = boxB
    wA, hA = wA * (1 + tolerance), hA * (1 + tolerance)
    wB, hB = wB * (1 + tolerance), hB * (1 + tolerance)
    x2A, y2A = x1A + wA, y1A + hA
    x2B, y2B = x1B + wB, y1B + hB
    xi1 = max(x1A, x1B)
    yi1 = max(y1A, y1B)
    xi2 = min(x2A, x2B)
    yi2 = min(y2A, y2B)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    boxA_area = wA * hA
    boxB_area = wB * hB
    union_area = boxA_area + boxB_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

In [ ]:
def evaluate_tracking(cvat_annotations, deepsort_detections, iou_threshold=0.1):
    tp, fp, fn = 0, 0, 0
    frames = set(cvat_annotations.keys()).union(deepsort_detections.keys())
    for frame in frames:
        gt_boxes = cvat_annotations.get(frame, [])
        pred_boxes = deepsort_detections.get(frame, [])
        matched = set()
        for gt in gt_boxes:
            for pred in pred_boxes:
                if iou(gt[1:], pred[1:]) >= iou_threshold:
                    matched.add(tuple(pred))
                    tp += 1
                    break

        fp += len(pred_boxes) - len(matched)
        fn += len(gt_boxes) - len(matched)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    accuracy = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
    return {"Precision": precision, "Recall": recall, "Accuracy": accuracy}

metrics = evaluate_tracking(cvat_annotations, deepsort_detections)
print(metrics)

{'Precision': 0.8163884673748103, 'Recall': 0.4769503546099291, 'Accuracy': 0.4307445956765412}


### Считаем показатели MOTP и MOTA

In [ ]:
import itertools

def evaluate_motp_mota(cvat_annotations, deepsort_detections, iou_threshold=0.1):
    tp, fp, fn = 0, 0, 0
    total_iou = 0
    id_switches = 0
    previous_matches = {}
    frames = set(cvat_annotations.keys()).union(deepsort_detections.keys())
    for frame in frames:
        gt_boxes = cvat_annotations.get(frame, [])
        pred_boxes = deepsort_detections.get(frame, [])
        matched = set()
        frame_matches = {}
        for gt in gt_boxes:
            gt_id, *gt_box = gt
            best_iou = 0
            best_pred = None
            for pred in pred_boxes:
                pred_id, *pred_box = pred
                iou_score = iou(gt_box, pred_box)
                if iou_score >= iou_threshold and iou_score > best_iou:
                    best_iou = iou_score
                    best_pred = pred
            if best_pred:
                matched.add(tuple(best_pred))
                tp += 1
                total_iou += best_iou
                frame_matches[gt_id] = best_pred[0]
                if gt_id in previous_matches and previous_matches[gt_id] != best_pred[0]:
                    id_switches += 1
        previous_matches = frame_matches
        fp += len(pred_boxes) - len(matched)
        fn += len(gt_boxes) - len(matched)
    motp = total_iou / tp if tp > 0 else 0
    mota = 1 - (fn + fp + id_switches) / sum(len(gt_boxes) for gt_boxes in cvat_annotations.values())
    return {"MOTP": motp, "MOTA": mota}

def optimize_hyperparameters(cvat_annotations, deepsort_detections):
    best_config = None
    best_mota, best_motp = 0, 0
    iou_thresholds = [0.1, 0.3, 0.5]
    max_ages = [10, 20, 30]
    min_hits_list = [2, 3, 5]
    nn_budgets = [50, 100, 150]
    for iou_threshold, max_age, min_hits, nn_budget in itertools.product(
        iou_thresholds, max_ages, min_hits_list, nn_budgets
    ):
        print(f"Тестируем: IoU={iou_threshold}, max_age={max_age}, min_hits={min_hits}, nn_budget={nn_budget}")

        metrics = evaluate_motp_mota(cvat_annotations, deepsort_detections, iou_threshold)
        mota, motp = metrics["MOTA"], metrics["MOTP"]

        print(f"Результат: MOTA={mota:.4f}, MOTP={motp:.4f}\n")

        if mota > best_mota or (mota == best_mota and motp > best_motp):
            best_mota, best_motp = mota, motp
            best_config = (iou_threshold, max_age, min_hits, nn_budget)

    print(f"\nЛучшая конфигурация: IoU={best_config[0]}, max_age={best_config[1]}, min_hits={best_config[2]}, nn_budget={best_config[3]}")
    print(f"Лучший результат: MOTA={best_mota:.4f}, MOTP={best_motp:.4f}")

    return best_config

best_hyperparams = optimize_hyperparameters(cvat_annotations, deepsort_detections)

Тестируем: IoU=0.1, max_age=10, min_hits=2, nn_budget=50
Результат: MOTA=0.2607, MOTP=0.4470

Тестируем: IoU=0.1, max_age=10, min_hits=2, nn_budget=100
Результат: MOTA=0.2607, MOTP=0.4470

Тестируем: IoU=0.1, max_age=10, min_hits=2, nn_budget=150
Результат: MOTA=0.2607, MOTP=0.4470

Тестируем: IoU=0.1, max_age=10, min_hits=3, nn_budget=50
Результат: MOTA=0.2607, MOTP=0.4470

Тестируем: IoU=0.1, max_age=10, min_hits=3, nn_budget=100
Результат: MOTA=0.2607, MOTP=0.4470

Тестируем: IoU=0.1, max_age=10, min_hits=3, nn_budget=150
Результат: MOTA=0.2607, MOTP=0.4470

Тестируем: IoU=0.1, max_age=10, min_hits=5, nn_budget=50
Результат: MOTA=0.2607, MOTP=0.4470

Тестируем: IoU=0.1, max_age=10, min_hits=5, nn_budget=100
Результат: MOTA=0.2607, MOTP=0.4470

Тестируем: IoU=0.1, max_age=10, min_hits=5, nn_budget=150
Результат: MOTA=0.2607, MOTP=0.4470

Тестируем: IoU=0.1, max_age=20, min_hits=2, nn_budget=50
Результат: MOTA=0.2607, MOTP=0.4470

Тестируем: IoU=0.1, max_age=20, min_hits=2, nn_budget=